In [1]:
import pandas as pd
import os

path = "/Users/royho/Desktop/World Bank Data - Datasets V2"
os.chdir(path)


In [2]:
#  2. Dataset filenames 
files = {
    "cpi": "Consumer price index (2010 = 100).csv",
    "food": "Conusmer Prices, Food Indices (2015 = 100).csv",
    "exchange": "Exchange Rates.csv",
    "gdp": "GDP (Current US$).csv",
    "inflation": "Inflation - Avg consumer prices index.csv",
    "population": "Population, total.csv"
}

dfs = {}

In [3]:
# 3. Load and clean 
for key, filename in files.items():
    print(f"\n🔹 Loading {filename}...")
    df = pd.read_csv(filename, low_memory=False)
    df.columns = df.columns.str.strip()

    # Handle V2 column pattern:
    # ['A', 'Annual', 'ALB', 'Albania', 'WB_WDI_FP_CPI_TOTL', 'Consumer price index (2010 = 100)', '1960', '1961', ...]
    if len(df.columns) < 8:
        print(f"⚠️ {filename} does not match expected structure. Skipping.")
        continue

    # Rename and keep relevant columns
    df.rename(
        columns={
            df.columns[2]: "Country Code",
            df.columns[3]: "Country Name"
        },
        inplace=True
    )

    # Extract year columns dynamically (start from column index 6 onward)
    year_cols = [col for col in df.columns if col.isdigit()]
    df = df[["Country Name", "Country Code"] + year_cols]

    # Melt into long format
    df = df.melt(id_vars=["Country Name", "Country Code"],
                 var_name="Year", value_name=key)
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
    dfs[key] = df

print("\n Loaded datasets:", list(dfs.keys()))


🔹 Loading Consumer price index (2010 = 100).csv...

🔹 Loading Conusmer Prices, Food Indices (2015 = 100).csv...

🔹 Loading Exchange Rates.csv...

🔹 Loading GDP (Current US$).csv...

🔹 Loading Inflation - Avg consumer prices index.csv...

🔹 Loading Population, total.csv...

 Loaded datasets: ['cpi', 'food', 'exchange', 'gdp', 'inflation', 'population']


In [4]:
#  4. Merge 
merged = dfs["cpi"]
for key in ["food", "gdp", "inflation", "exchange", "population"]:
    if key in dfs:
        merged = pd.merge(merged, dfs[key], on=[
                          "Country Name", "Country Code", "Year"], how="outer")

In [5]:
#  5. Filter for North American countries 
countries = ["United States", "Canada", "Mexico"]
merged = merged[merged["Country Name"].isin(countries)]

In [6]:
#6. Sort, fill, save
merged.sort_values(by=["Country Name", "Year"], inplace=True)
merged = merged.groupby("Country Name").ffill().bfill()

output_path = os.path.join(path, "Cleaned_WorldBank_FoodInflation.csv")
merged.to_csv(output_path, index=False)


In [7]:
print("\nCleaning done")
print(f"Saved to: {output_path}")
print("\nPreview:")
print(merged.head())


Cleaning done
Saved to: /Users/royho/Desktop/World Bank Data - Datasets V2/Cleaned_WorldBank_FoodInflation.csv

Preview:
      Country Code  Year        cpi  food           gdp  inflation  exchange  \
25520          CAN  1940  13.344305   NaN  4.056377e+10     44.037  0.918274   
25521          CAN  1940  13.344305   NaN  4.056377e+10     44.037  0.918274   
25522          CAN  1940  13.344305   NaN  4.056377e+10     44.037  0.918274   
25523          CAN  1940  13.344305   NaN  4.056377e+10     44.037  0.918274   
25524          CAN  1940  13.344305   NaN  4.056377e+10     44.037  0.918274   

       population  
25520  17909356.0  
25521  17909356.0  
25522  17909356.0  
25523  17909356.0  
25524  17909356.0  
